In [22]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
def pass_to_markdown(file_path):

    file_name = os.path.basename(file_path)
    
    file_name = os.path.join("data_md", file_name)

    loader = DoclingLoader(
            file_path=file_path,
            export_type=ExportType.MARKDOWN
            )

    docs = loader.load()

    clean_doc = docs[0].page_content.replace("glyph&lt;c=3,font=/CIDFont+F8&gt;", "").replace("glyph&lt;c=3,font=/CIDFont+F5&gt;", "").replace("glyph<c=3,font=/CIDFont+F5>", " ").replace("glyph<c=3,font=/CIDFont+F8>", " ")

    with open(f"{file_name}_md.md", "w", encoding="utf-8") as markdown_file:
        markdown_file.write(clean_doc)
    return f"{file_name}_md.md"

In [23]:
import json
def clean_docs(file_path):

    def process_row(line):
        # Remove espaços extras entre o conteúdo da célula e os pipes
        cells = [cell.strip() for cell in line.split('|')[1:-1]]  # Ignora elementos vazios no início/fim
        return '| ' + ' | '.join(cells) + ' |'  # Reconstroi a linha com 1 espaço entre texto e "|"
    
    # Carregar o JSON com as correções
    with open('miss_words.json', 'r', encoding='utf-8') as f:
        correcoes = json.load(f)["replacements"]

    # Ler o documento original
    with open(file_path, 'r', encoding='utf-8') as f:
        texto = f.readlines()

    texto_corrigido = ''

    in_table = False

    for line in texto:
        for erro, correcao in correcoes.items():
            line = line.replace(erro, correcao)

        stripped_line = line.strip()

        # Verifica se é uma linha de tabela
        if stripped_line.startswith('|') and stripped_line.endswith('|'):
            if not in_table:
                in_table = True
            # Processa a linha para normalizar espaços
            processed_line = process_row(stripped_line)
            texto_corrigido += processed_line + "\n"
        else:
            texto_corrigido += stripped_line + "\n"
            if in_table:
                in_table = False
        

    file_name = os.path.basename(file_path)
    
    file_name = os.path.join("data_md_correct", file_name)

    # Salvar o documento corrigido
    with open(f"{file_name}_correct.md", 'w', encoding='utf-8') as f:
        f.write(texto_corrigido)
    return file_name

In [24]:
def extract_markdown_tables(file_path):
    def process_row(line):
        # Remove espaços extras entre o conteúdo da célula e os pipes
        cells = [cell.strip() for cell in line.split('|')[1:-1]]  # Ignora elementos vazios no início/fim
        return '| ' + ' | '.join(cells) + ' |'  # Reconstroi a linha com 1 espaço entre texto e "|"
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    tables = []
    current_table = []
    in_table = False
    
    for line in lines:
        stripped_line = line.strip()
        # Verifica se é uma linha de tabela
        if stripped_line.startswith('|') and stripped_line.endswith('|'):
            if not in_table:
                in_table = True
            # Processa a linha para normalizar espaços
            processed_line = process_row(stripped_line)
            current_table.append(processed_line)
        else:
            if in_table:
                if current_table:
                    tables.append(current_table)
                    current_table = []
                in_table = False
    
    # Adiciona a última tabela se o arquivo terminar com uma tabela
    if in_table and current_table:
        tables.append(current_table)
    
    return tables

# Exemplo de uso:
# start_file_path = 'clean_doc.md'

# end_file_path = clean_docs(start_file_path)

# tables = extract_markdown_tables(end_file_path)

# Imprime as tabelas encontradas
# for i, table in enumerate(tables, 1):
#     print(f"Tabela {i}:")
#     for row in table:
#         print(row)
#     print("\n" + "-"*50 + "\n")

In [ ]:
import os

for i, file in enumerate(os.listdir('data')):
    
    start_file_path = 'data/'+file
    print(start_file_path)

    markdown_path = pass_to_markdown(start_file_path)
    print(markdown_path)

    end_file_path = clean_docs(markdown_path)
    print(end_file_path)
    # tables = extract_markdown_tables(end_file_path)

LOOP: 0
data/citros.pdf
PROFAVOR DA SERTO data_md/citros.pdf
data_md/citros.pdf_md.md
data_md_correct/citros.pdf_md.md
